In [ ]:
# import heterodata

In [ ]:
def train_loop():
    epoch_0 = 0
    for epoch in range(epoch_0, self.autoencoder_num_epochs):
                epoch_loss = []
                print("Experiment: autoencoder {0}: training Epoch = ".format(self.autoencoder), epoch+1, 'out of', self.autoencoder_num_epochs, 'epochs')
    
                # Loop through all the train data using the data loader
                for ii, (dem, ob, ac, l, t, scores, rewards, idx) in enumerate(self.train_loader):
                    # print("Batch {}".format(ii),end='')
                    dem = dem.to(device)  # 5 dimensional vector (Gender, Ventilation status, Re-admission status, Age, Weight)
                    ob = ob.to(device)    # 33 dimensional vector (time varying measures)
                    ac = ac.to(device) # actions
                    l = l.to(device)
                    t = t.to(device)
                    scores = scores.to(device)
                    idx = idx.to(device)
                    loss_pred = 0
    
                    # Cut tensors down to the batch's largest sequence length... Trying to speed things up a bit...
                    max_length = int(l.max().item())
    
                    # The following losses are for DDM and will not be modified by any other approach
                    train_loss, dec_loss, inv_loss = 0, 0, 0
                    model_loss, recon_loss, forward_loss = 0, 0, 0                    
                        
                    # Set training mode (nn.Module.train()). It does not actually trains the model, but just sets the model to training mode.
                    self.gen.train()
                    self.pred.train()
    
                    ob = ob[:,:max_length,:]
                    dem = dem[:,:max_length,:]
                    ac = ac[:,:max_length,:]
                    scores = scores[:,:max_length,:]
                    
                    # Special case for CDE
                    # Getting loss_pred and mse_loss
                    if self.autoencoder == 'CDE':
                        loss_pred, mse_loss, _ = self.container.loop(ob, dem, ac, scores, l, max_length, self.context_input, corr_coeff_param = self.corr_coeff_param, device = device, coefs = self.train_coefs, idx = idx)
                    else:
                        loss_pred, mse_loss, _ = self.container.loop(ob, dem, ac, scores, l, max_length, self.context_input, corr_coeff_param = self.corr_coeff_param, device=device, autoencoder = self.autoencoder)   
    
                    self.optimizer.zero_grad()
                    
                    if self.autoencoder != 'DDM':
                        loss_pred.backward()
                        self.optimizer.step()
                        epoch_loss.append(loss_pred.detach().cpu().numpy())                
                    else:
                        # For DDM
                        train_loss, dec_loss, inv_loss, model_loss, recon_loss, forward_loss, corr_loss, loss_pred = loss_pred
                        train_loss = forward_loss + self.inv_loss_coef*inv_loss + self.dec_loss_coef*dec_loss - self.corr_coeff_param*corr_loss.sum()
                        train_loss.backward()
                        # Clipping gradients to prevent exploding gradients
                        torch.nn.utils.clip_grad_norm(self.all_params, self.max_grad_norm)
                        self.optimizer.step()
                        epoch_loss.append(loss_pred.detach().cpu().numpy())
                                            
                self.autoencoding_losses.append(epoch_loss)
                if (epoch+1)%self.saving_period == 0: # Run validation and also save checkpoint
                    
                    #Computing validation loss
                    epoch_validation_loss = []
                    with torch.no_grad():
                        for jj, (dem, ob, ac, l, t, scores, rewards, idx) in enumerate(self.val_loader):
    
                            dem = dem.to(device)
                            ob = ob.to(device)
                            ac = ac.to(device)
                            l = l.to(device)
                            t = t.to(device)
                            idx = idx.to(device)
                            scores = scores.to(device)
                            loss_val = 0
    
                            # Cut tensors down to the batch's largest sequence length... Trying to speed things up a bit...
                            max_length = int(l.max().item())                        
                            
                            ob = ob[:,:max_length,:]
                            dem = dem[:,:max_length,:]
                            ac = ac[:,:max_length,:] 
                            scores = scores[:,:max_length,:] 
                            
                            self.gen.eval()
                            self.pred.eval()    
                            
                            if self.autoencoder == 'CDE':
                                loss_val, mse_loss, _ = self.container.loop(ob, dem, ac, scores, l, max_length, corr_coeff_param = 0, device = device, coefs = self.val_coefs, idx = idx)
                            else:
                                loss_val, mse_loss, _ = self.container.loop(ob, dem, ac, scores, l, max_length, self.context_input, corr_coeff_param = 0, device=device, autoencoder = self.autoencoder)                                                 
                            
                            if self.autoencoder in ['DST', 'ODERNN', 'CDE']:
                                epoch_validation_loss.append(mse_loss)
                            elif self.autoencoder == "DDM":
                                epoch_validation_loss.append(loss_val[-1].detach().cpu().numpy())
                            else:
                                epoch_validation_loss.append(loss_val.detach().cpu().numpy())
                        
                            
                    self.autoencoding_losses_validation.append(epoch_validation_loss)
    
                    save_dict = {'epoch': epoch,
                            'gen_state_dict': self.gen.state_dict(),
                            'pred_state_dict': self.pred.state_dict(),
                            'optimizer_state_dict': self.optimizer.state_dict(),
                            'loss': self.autoencoding_losses,
                            'validation_loss': self.autoencoding_losses_validation
                            }
                    
                    if self.autoencoder == 'DDM':
                        save_dict['dyn_state_dict'] = self.dyn.state_dict()
                        
                    try:
                        torch.save(save_dict, self.checkpoint_file)
                        # torch.save(save_dict, self.checkpoint_file[:-3] + str(epoch) +'_.pt')
                        np.save(self.data_folder + '/{}_losses.npy'.format(self.autoencoder.lower()), np.array(self.autoencoding_losses))
                    except Exception as e:
                        print(e)
    
                    
                    try:
                        np.save(self.data_folder + '/{}_validation_losses.npy'.format(self.autoencoder.lower()), np.array(self.autoencoding_losses_validation))
                    except Exception as e:
                        print(e)
                        
                #Final epoch checkpoint
                try:
                    save_dict = {
                                'epoch': self.autoencoder_num_epochs-1,
                                'gen_state_dict': self.gen.state_dict(),
                                'pred_state_dict': self.pred.state_dict(),
                                'optimizer_state_dict': self.optimizer.state_dict(),
                                'loss': self.autoencoding_losses,
                                'validation_loss': self.autoencoding_losses_validation,
                                }
                    if self.autoencoder == 'DDM':
                        save_dict['dyn_state_dict'] = self.dyn.state_dict()
                        torch.save(self.dyn.state_dict(), self.dyn_file)
                    torch.save(self.gen.state_dict(), self.gen_file)
                    torch.save(self.pred.state_dict(), self.pred_file)
                    torch.save(save_dict, self.checkpoint_file)
                    np.save(self.data_folder + '/{}_losses.npy'.format(self.autoencoder.lower()), np.array(self.autoencoding_losses))
                except Exception as e:
                        print(e)
               

In [ ]:
dataset.node_time_dict = {
    "user": torch.tensor([1, 3, 5, 7]),  # Timestamps for user nodes
    "item": torch.tensor([2, 4, 6, 8]),  # Timestamps for item nodes
}

In [ ]:
# @timeit
def time_select_up_to_t(dataset, t):
    """Select nodes and edges up to and including time step t.
    @param t: time index (inclusive)
    @return: HeteroData, with sliced nodes and edges based on time.
    """
    dt = HeteroData()
    d = dataset

    # Get node timestamps (if they exist)
    node_time_dict = getattr(d, "node_time_dict", {})  # Use empty dict if node_time_dict doesn't exist

    # Copy node information up to and including time step t
    for ntype, value in d.x_dict.items():
        if ntype in node_time_dict:  # If node type has time information
            mask = (node_time_dict[ntype] <= t).squeeze(-1)  # Filter nodes based on time
            dt[ntype].x = value[mask]  # Copy node features for filtered nodes
            if "num_nodes" in d[ntype].keys():
                dt[ntype].num_nodes = mask.sum().item()  # Update number of nodes
        else:  # If node type has no time information, copy all nodes
            dt[ntype].x = value
            if "num_nodes" in d[ntype].keys():
                dt[ntype].num_nodes = d[ntype].num_nodes

    # Get edge timestamps and edge indices
    dea = d.edge_time_dict
    dei = d.edge_index_dict

    # Filter edges up to and including time step t
    for etype in dea:
        mask = (dea[etype] <= t).squeeze(-1)  # Include all edges with time <= t
        dt[etype].edge_index = dei[etype][:, mask]
        dt[etype].edge_time = dea[etype][mask]

    return dt